# Week 06 | Siraj Raval | MMWML

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [2]:
!pip install tensorflow-gpu==1.14
clear_output()

import tensorflow as tf

tf.__version__

'1.14.0'

## Homework Statement

The homework for this week is to create a Generative Adversarial Network that will be able to generate novel images after training. The dataset to be used for this is [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist).

## 0 References

### 0.1 Project Help

- Rowel Atienza's [book](https://www.amazon.com/Advanced-Deep-Learning-Keras-reinforcement/dp/1788629418/) and [Medium article](https://towardsdatascience.com/gan-by-example-using-keras-on-tensorflow-backend-1a6d515a60d0) provided my fundamental understanding of implementation of a simple Deep Convolutional GAN (DC-GAN)
- Rowel also attached a sample python [script](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/Experiments/Tensorflow/GAN/dcgan_mnist.py) in the Medium article referenced above which had a template of the code

- Github User `R-Suresh`'s [code](https://github.com/R-Suresh/GAN_fashion_MNIST/blob/master/gan.py)

### 0.2 Alternative to MNIST dataset: Fashion-MNIST

___Introducing the Fashion-MNIST___

[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) is a dataset of [Zalando's](https://jobs.zalando.com/en/) article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in replacement for the [original MNIST dataset](http://yann.lecun.com/exdb/mnist/) for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

<img src = "https://tensorflow.org/images/fashion-mnist-sprite.png" width = 450 height = 400>

<br>

___Why was the Fashion-MNIST created?___

The original MNIST dataset contains a lot of handwritten digits. Members of the AI/ML/Data Science community love this dataset and use it as a benchmark to validate their algorithms. In fact, MNIST is often the first dataset researchers try. _"If it doesn't work on MNIST, it won't work at all"_ , they said. 

_"Well, if it does work on MNIST, it may still fail on others."_

<img src = "https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/embedding.gif">

Here are some good reasons to replace MNIST:

- __MNIST is too easy__: Convolutional nets can achieve 99.7% on MNIST. Classic machine learning algorithms can also achieve 97% easily.
- __MNIST is overused__: In [this April 2017 Twitter thread](https://twitter.com/goodfellow_ian/status/852591106655043584), Google Brain research scientist and deep learning expert Ian Goodfellow calls for people to move away from MNIST.
- __MNIST can not represent modern CV tasks__, as noted in [this April 2017 Twitter thread](https://twitter.com/fchollet/status/852594987527045120), deep learning expert/Keras author François Chollet.

In [3]:
from tensorflow.keras.datasets.fashion_mnist import load_data

(x_train, y_train), (x_test, y_test) = load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# shape of the image datasets
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [5]:
# shape of the label vectors
y_train.shape, y_test.shape

((60000,), (10000,))

In [6]:
# values in the label vectors

class_ids = np.unique(y_train)
class_ids

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

___Actual labels for the numbers in `class_ids`___

| 0 | T-shirt/top |
|:-:|:-----------:|
| 1 |   Trouser   |
| 2 |   Pullover  |
| 3 |    Dress    |
| 4 |     Coat    |
| 5 |    Sandal   |
| 6 |    Shirt    |
| 7 |   Sneaker   |
| 8 |     Bag     |
| 9 |  Ankle Boot |

## 1 Creating a GAN model using `tf.keras`

In [0]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU

import sys

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        #model.add(Dense(2048))
        #model.add(LeakyReLU(alpha=0.2))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        #model.add(Dense(512))
        #model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        #fig.savefig("images/%d.png" % epoch)
        plt.show()
        plt.close()

## 2 Training the GAN model

In [0]:
def train(GAN_object, epochs, batch_size = 256, sample_interval = 50):

        # Rescale -1 to 1
        X_train = x_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, GAN_object.latent_dim))

            # Generate a batch of new images
            gen_imgs = GAN_object.generator.predict(noise)

            # Train the discriminator
            d_loss_real = GAN_object.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = GAN_object.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, GAN_object.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = GAN_object.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                GAN_object.sample_images(epoch)

In [0]:
GAN_object = GAN()
train(GAN_object, epochs = 3000, batch_size = 256, sample_interval = 200)

`2990 [D loss: 0.641810, acc.: 62.30%] [G loss: 0.823704]`

`2991 [D loss: 0.659239, acc.: 59.38%] [G loss: 0.845749]`

`2992 [D loss: 0.666248, acc.: 58.01%] [G loss: 0.801034]`

`2993 [D loss: 0.659432, acc.: 58.20%] [G loss: 0.825578]`

`2994 [D loss: 0.657774, acc.: 58.20%] [G loss: 0.826891]`

`2995 [D loss: 0.669790, acc.: 55.66%] [G loss: 0.825897]`

`2996 [D loss: 0.675206, acc.: 52.73%] [G loss: 0.828771]`

`2997 [D loss: 0.659878, acc.: 59.57%] [G loss: 0.840454]`

`2998 [D loss: 0.664810, acc.: 59.57%] [G loss: 0.848437]`

`2999 [D loss: 0.667068, acc.: 60.16%] [G loss: 0.835160]`